
INSTRUCTIONS: Just Upload the file to Jupyter Notebook or Google Colab and run each cell one after the other                                      
NOTE: For Jupyter notebook, you will have to make sure you have the relevant packages and libraries imported below downloaded into you Anaconda environment.

In [1]:
import os
import warnings
import re
warnings.filterwarnings("ignore", category=UserWarning)

from collections import defaultdict

import pandas as pd
import numpy as np

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

In [2]:
# Download dataset
PROJECT_DIR = os.getcwd() + '/afrisent-semeval-2023'
print('Current directiory: ', PROJECT_DIR)
PROJECT_GITHUB_URL = 'https://github.com/afrisenti-semeval/afrisent-semeval-2023.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  !git pull {PROJECT_GITHUB_URL}

Current directiory:  C:\Users\ephra\Documents\UCT Jupyter Notebook\NLP/afrisent-semeval-2023
C:\Users\ephra\Documents\UCT Jupyter Notebook\NLP\afrisent-semeval-2023
Already up to date.


From https://github.com/afrisenti-semeval/afrisent-semeval-2023
 * branch            HEAD       -> FETCH_HEAD


In [3]:
language = 'twi'

In [4]:
# Load data
DATA_DIR = f'{PROJECT_DIR}/data/{language}'
print('Data directory: ', DATA_DIR)

train_df = pd.read_csv(f'{DATA_DIR}/train.tsv', sep='\t', names=['text', 'label'], header=0)
dev_df = pd.read_csv(f'{DATA_DIR}/dev.tsv', sep='\t', names=['text', 'label'], header=0)
test_df = pd.read_csv(f'{DATA_DIR}/test.tsv', sep='\t', names=['text', 'label'], header=0)

Data directory:  C:\Users\ephra\Documents\UCT Jupyter Notebook\NLP/afrisent-semeval-2023/data/twi


In [5]:
# Data Shape
print('Train shape: ', train_df.shape)
print('Dev shape: ', dev_df.shape)
print('Test shape: ', test_df.shape)

Train shape:  (3481, 2)
Dev shape:  (388, 2)
Test shape:  (949, 2)


In [6]:
# Display data
train_df.sample(10)

,text,label
2235,3herh oh wo yaw sarpong ankasa ni s3n wo haw n...,positive
1025,wei na wo di p3 sporting abo no 🤣,negative
1666,woyem ahye wo wnnnn,neutral
416,ring koraa yedi na header madi wo awie 😂,negative
3458,ne nyinaa biy3b3di sallah no paa,positive
899,me yem koraa hyehye me 😭😭😭,negative
2147,wode33 fa ps5 sika no twa me surprise,positive
3197,sister debbie is pretty 🤩 eii meyem kraa ahye me,positive
2417,woa wasa nsuo kakra aka no wo san de awarɛ asɛ...,positive
402,ei asi enti min ka ho bi,negative


In [7]:
# Discard neutral examples
train_df = train_df[train_df['label'] != 'neutral']
dev_df = dev_df[dev_df['label'] != 'neutral']
test_df = test_df[test_df['label'] != 'neutral']

In [8]:
train_df.sample(20)

,text,label
2390,aseey aba time na album nu b3 drop na ma tor d...,positive
659,that guy nyansa baako sei wo nni bi,negative
532,wo ti awu forken wo,negative
3134,yɛ digi online gang keep the secrets in mind g...,positive
1291,nyame b3tua klopp ka papaaapa,negative
1186,nyansa doduor yɛ gyimie 🤣🤣🤣,negative
830,ahhhh kumasi foɔ ne jon deɛ😹😹😹,negative
3263,ma me y3 wa danfo err,positive
974,aaaah na wo brofo left left bɛn nni,negative
2887,agye wonmo nsamu ntiaa but dont worry cause 3n...,positive


In [9]:
def clean(text):
    # Replace URLS with [URL]
    text = re.sub(r'http\S+', '[URL]', text)
    
    # Replace numbers with [NUM]
    text = re.sub(r'\d+', '[NUM]', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove emojis and special characters
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    
    # Ensure there are no multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove trailing spaces
    text = text.strip()
    
    return text

In [10]:
# Clean the text in the dataframes
train_df['text'] = train_df['text'].apply(clean)
dev_df['text'] = dev_df['text'].apply(clean)
test_df['text'] = test_df['text'].apply(clean)

BITE PAIR ENCODING TOKENIZATION CLASS

In [11]:
class BPETokenizer():
    def __init__(self, sentences, vocab_size):
        self.sentences = sentences
        self.vocab_size = vocab_size
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}

        # Initialize word frequencies
        for sentence in self.sentences:
            for word in sentence:
                self.word_freqs[word] += 1

    def train(self):
        for word in self.word_freqs:
            self.splits[word] = [char for char in word]
        
        while len(self.splits) < self.vocab_size:
            pair_freqs = self.compute_pair_freqs()
            if not pair_freqs:
                break
            best_pair = max(pair_freqs, key=pair_freqs.get)
            self.merge_pair(best_pair[0], best_pair[1])
        
        return self.merges

    def compute_pair_freqs(self):
        pairs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            symbols = self.splits[word]
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs

    def merge_pair(self, a, b):
        merge = a + b
        for word in list(self.splits):
            split = self.splits[word]
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [merge] + split[i + 2:]
                else:
                    i += 1
            self.splits[word] = split
        self.merges[(a, b)] = merge
        return self.splits

    def tokenize(self, text):
        pre_tokenized_text = text.split()
        splits_text = [[char for char in word] for word in pre_tokenized_text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2:]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result


In [12]:
# Example usage
train_corpus = [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence']]
bpe_tokenizer = BPETokenizer(train_corpus, vocab_size=50)
bpe_tokenizer.train()
print(bpe_tokenizer.tokenize("This is a sentence"))

['This', 'is', 'a', 'sentence']


TRAINING BITE PAIR TOKENIZATION MODEL

In [13]:
# Train BPE
train_corpus = train_df['text'].tolist()
tokenized_train_corpus = [list(word) for sentence in train_corpus for word in sentence.split()]

bpe = BPETokenizer(tokenized_train_corpus, vocab_size=1000)
merges = bpe.train()

# Apply BPE tokenization to our dataset
train_df['bpe_text'] = train_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))
dev_df['bpe_text'] = dev_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))
test_df['bpe_text'] = test_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))

In [14]:
import pickle
# Save merges to a file after training
with open('bpe_merges.pkl', 'wb') as f:
    pickle.dump(merges, f)

In [15]:
train_df.sample(10)

,text,label,bpe_text
2118,masamasa bolebamboi ooo kumasi ooo theyve take...,positive,m a s a m a s a b o l e b a m b o i o o o k u ...
2366,reality check be what lol ynumti sika so numnso,positive,r e a l i t y c h e c k b e w h a t l o l y n ...
3476,kasa gu ma sum na me gyedi,positive,k a s a g u m a s u m n a m e g y e d i
1051,wei wei y de gye cup,negative,w e i w e i y d e g y e c u p
2782,numka wo nantini a na aka wo to,positive,n u m k a w o n a n t i n i a n a a k a w o t o
2537,ynum ynum dom wo kum apem a apem bnumba king p...,positive,y n u m y n u m d o m w o k u m a p e m a a p ...
2172,sika no asa sesiaa,positive,s i k a n o a s a s e s i a a
779,wo mpo wo te kurom mpo na wagyimi saa no numne...,negative,w o m p o w o t e k u r o m m p o n a w a g y ...
910,theyre all de same bro nkwasiafo nkoaaaa,negative,t h e y r e a l l d e s a m e b r o n k w a s ...
194,buy wig na ti ti fa tari h,negative,b u y w i g n a t i t i f a t a r i h


In [16]:
# Check if there are empty documents
empty_docs = train_df[train_df['bpe_text'].str.strip().eq('')]
print(f"Number of empty documents in train set: {empty_docs.shape[0]}")

Number of empty documents in train set: 0


In [17]:
text1 = 'Omar is waiting for me'
token1 = bpe.tokenize(text1)
print(token1)

['O', 'm', 'a', 'r', 'i', 's', 'w', 'a', 'i', 't', 'i', 'n', 'g', 'f', 'o', 'r', 'm', 'e']


WHITE SPACE TOKENIZATION CLASS

In [18]:
class WordTokenizer():
    def __init__(self):
        pass

    def tokenize(self, text):
        
        return text.split()  # Splits on whitespace by default

In [19]:
# Example usage
word_tokenizer = WordTokenizer()
text = 'This is a test sentence.'
tokens = word_tokenizer.tokenize(text)
print('Word-based tokens:', tokens)

Word-based tokens: ['This', 'is', 'a', 'test', 'sentence.']


FEATURE EXTRACTION FROM TOKENIZATION

In [20]:
# Importing sklearn libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [21]:
# Feature extraction using count-based features for word-based tokenization
word_vectorizer = CountVectorizer()
X_train_word = word_vectorizer.fit_transform(train_df['text'])
X_dev_word = word_vectorizer.transform(dev_df['text'])
X_test_word = word_vectorizer.transform(test_df['text'])

In [22]:
print(X_train_word)

  (0, 2461)	1
  (0, 757)	1
  (0, 4579)	1
  (0, 1068)	1
  (0, 5403)	1
  (0, 4937)	1
  (0, 1754)	1
  (1, 1005)	1
  (1, 3455)	1
  (1, 747)	1
  (1, 3711)	1
  (1, 3386)	1
  (1, 2994)	1
  (1, 4826)	1
  (1, 2904)	1
  (1, 1180)	1
  (2, 5403)	1
  (2, 4681)	2
  (2, 5513)	1
  (2, 2854)	1
  (2, 5216)	1
  (2, 4862)	1
  (2, 3632)	1
  (2, 3893)	1
  (2, 4687)	1
  :	:
  (2955, 3754)	1
  (2955, 486)	1
  (2955, 2497)	1
  (2956, 232)	3
  (2956, 4691)	1
  (2956, 2243)	1
  (2956, 4764)	1
  (2956, 3389)	1
  (2956, 4851)	1
  (2956, 2675)	1
  (2956, 4204)	1
  (2957, 3711)	1
  (2957, 3229)	1
  (2957, 553)	1
  (2957, 3816)	1
  (2957, 2608)	1
  (2957, 905)	1
  (2958, 3386)	1
  (2958, 3632)	1
  (2958, 3340)	1
  (2958, 1591)	1
  (2958, 3385)	1
  (2958, 5379)	1
  (2958, 3841)	1
  (2958, 3100)	1


In [23]:
# Feature extraction using count-based features for cleaned BPE tokenization
# bpe_vectorizer = CountVectorizer(stop_words=None)
bpe_vectorizer = CountVectorizer(stop_words=None, token_pattern=r'\b\w+\b')

X_train_bpe = bpe_vectorizer.fit_transform(train_df['bpe_text'])
X_dev_bpe = bpe_vectorizer.transform(dev_df['bpe_text'])
X_test_bpe = bpe_vectorizer.transform(test_df['bpe_text'])

In [24]:
print(X_train_bpe)

  (0, 10)	3
  (0, 0)	2
  (0, 14)	2
  (0, 1)	2
  (0, 4)	2
  (0, 18)	1
  (0, 7)	1
  (0, 17)	1
  (0, 20)	2
  (0, 19)	2
  (0, 22)	2
  (0, 8)	1
  (1, 0)	3
  (1, 14)	1
  (1, 1)	2
  (1, 4)	3
  (1, 18)	1
  (1, 17)	2
  (1, 20)	1
  (1, 19)	3
  (1, 8)	4
  (1, 13)	4
  (1, 24)	2
  (1, 12)	2
  (1, 15)	2
  :	:
  (2956, 3)	1
  (2957, 10)	1
  (2957, 0)	4
  (2957, 14)	2
  (2957, 1)	1
  (2957, 4)	1
  (2957, 18)	2
  (2957, 20)	3
  (2957, 19)	1
  (2957, 8)	1
  (2957, 13)	4
  (2957, 24)	1
  (2957, 12)	4
  (2957, 6)	1
  (2958, 0)	1
  (2958, 14)	3
  (2958, 4)	4
  (2958, 20)	2
  (2958, 22)	1
  (2958, 8)	3
  (2958, 13)	5
  (2958, 24)	1
  (2958, 12)	5
  (2958, 15)	2
  (2958, 5)	1


DEFINING LABLES FOR THE DATA

In [25]:
# Labels
y_train = train_df['label']
y_dev = dev_df['label']
y_test = test_df['label']

In [31]:
print(y_train)

0       negative
1       negative
2       negative
3       negative
4       negative
          ...   
3476    positive
3477    positive
3478    positive
3479    positive
3480    positive
Name: label, Length: 2959, dtype: object


In [32]:
# Train Naive Bayes classifier for word-based tokenization with Laplace smoothing
nb_word = MultinomialNB(alpha=1.0)
nb_word.fit(X_train_word, y_train)
y_pred_dev_word = nb_word.predict(X_dev_word)
print("Word-based Tokenization - Classification Report on Dev Set")
print(classification_report(y_dev, y_pred_dev_word))

Word-based Tokenization - Classification Report on Dev Set
              precision    recall  f1-score   support

    negative       0.73      0.76      0.74       147
    positive       0.80      0.78      0.79       183

    accuracy                           0.77       330
   macro avg       0.76      0.77      0.76       330
weighted avg       0.77      0.77      0.77       330



In [41]:
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\ephra\Documents\UCT Jupyter Notebook\NLP\afrisent-semeval-2023


In [48]:
predictions = nb_word.predict(X_dev_word)

output_file = 'dev_set_predictions_word.txt'

with open(output_file, 'w') as f:
    for text, prediction in zip(dev_df['text'], predictions):
        f.write(f"Sentence: {text}\nPredicted Sentiment: {prediction}\n\n")

print(f"Predictions saved to {output_file}")

Predictions saved to dev_set_predictions_word.txt


In [39]:
# Train Naive Bayes classifier for BPE tokenization with Laplace smoothing
nb_bpe = MultinomialNB(alpha=1.0)
nb_bpe.fit(X_train_bpe, y_train)
y_pred_dev_bpe = nb_bpe.predict(X_dev_bpe)
print("BPE Tokenization - Classification Report on Dev Set")
print(classification_report(y_dev, y_pred_dev_bpe))

BPE Tokenization - Classification Report on Dev Set
              precision    recall  f1-score   support

    negative       0.59      0.46      0.52       147
    positive       0.63      0.75      0.69       183

    accuracy                           0.62       330
   macro avg       0.61      0.60      0.60       330
weighted avg       0.61      0.62      0.61       330



In [49]:
predictions = nb_bpe.predict(X_dev_bpe)

output_file = 'dev_set_predictions_bpe.txt'

with open(output_file, 'w') as f:
    for text, prediction in zip(dev_df['text'], predictions):
        f.write(f"Sentence: {text}\nPredicted Sentiment: {prediction}\n\n")

print(f"Predictions saved to {output_file}")

Predictions saved to dev_set_predictions_bpe.txt


In [193]:
# Prediction and evaluation on the test set for word-based tokenization
y_pred_test_word = nb_word.predict(X_test_word)
print("Word-based Tokenization - Classification Report on Test Set")
print(classification_report(y_test, y_pred_test_word))

Word-based Tokenization - Classification Report on Test Set
              precision    recall  f1-score   support

    negative       0.76      0.76      0.76       353
    positive       0.81      0.81      0.81       450

    accuracy                           0.79       803
   macro avg       0.79      0.79      0.79       803
weighted avg       0.79      0.79      0.79       803



In [194]:
# Prediction and evaluation on the test set for BPE tokenization
y_pred_test_bpe = nb_bpe.predict(X_test_bpe)
print("BPE Tokenization - Classification Report on Test Set")
print(classification_report(y_test, y_pred_test_bpe))

BPE Tokenization - Classification Report on Test Set
              precision    recall  f1-score   support

    negative       0.57      0.46      0.51       353
    positive       0.63      0.73      0.68       450

    accuracy                           0.61       803
   macro avg       0.60      0.59      0.59       803
weighted avg       0.60      0.61      0.60       803



PREDICTION FUNCTION TO CLASSIFY NEW SENTENCES AS EITHER POSITIVE OR NEGATIVE

In [233]:
class BPETokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.merges = {}

    def load_merges(self, merges_file):
        with open(merges_file, 'rb') as f:
            self.merges = pickle.load(f)

    def tokenize(self, text):
        pre_tokenized_text = text.split()
        splits_text = [[l for l in word] for word in pre_tokenized_text]
        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2:]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result

def predict_sentiment_bpe(text, model, vectorizer, tokenizer):
    # Load merges
    tokenizer.load_merges('bpe_merges.pkl')

    # Clean and tokenize the input text using BPE
    clean_text = clean(text)
    tokenized_text = tokenizer.tokenize(clean_text)
    
    # Convert tokens to feature vectors using the vectorizer
    features = vectorizer.transform([' '.join(tokenized_text)])
    
    # Make prediction
    prediction = model.predict(features)
    
    return prediction[0]

In [239]:
# Example usage for BPE tokenization
tokenizer = BPETokenizer(vocab_size=1000)  # Initialize tokenizer with the desired vocab size
text_bpe = "wo ye gyimi paa"
predicted_sentiment_bpe = predict_sentiment_bpe(text_bpe, nb_bpe, bpe_vectorizer, tokenizer)
print(f"Predicted sentiment (BPE): {predicted_sentiment_bpe}")

Predicted sentiment (BPE): negative


In [248]:
word_tokenizer = WordTokenizer()

def predict_sentiment_word_based(text, model, vectorizer):

    # Clean the input text
    clean_text = clean(text)

    # Tokenize the cleaned text using WordTokenizer
    tokenized_text = word_tokenizer.tokenize(clean_text)

    # Convert tokenized text to feature vectors
    features = vectorizer.transform([' '.join(tokenized_text)])

    # Make prediction
    prediction = model.predict(features)
    
    return prediction[0]

In [250]:
# Example usage
text_word_based = "me pe wo wate"
predicted_sentiment_word_based = predict_sentiment_word_based(text_word_based, nb_word, word_vectorizer)
print(f"Predicted sentiment (Word-Based): {predicted_sentiment_word_based}")

Predicted sentiment (Word-Based): positive
